In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.utils import plot_model

In [ ]:
# Building Quality Model
def build_quality_model(input_shape=(128, 128, 3)):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  
    ])
    return model

In [ ]:
# Draw the Chart
plot_model(build_quality_model, to_file='quality_model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Fake data
df = pd.DataFrame({
    'bedrooms': np.random.randint(1, 6, 100),
    'bathrooms': np.random.randint(1, 4, 100),
    'solar': np.random.randint(0, 2, 100),
    'AC': np.random.randint(0, 2, 100),
    'pool': np.random.randint(0, 2, 100),
    'avg_nearby_price': np.random.uniform(200000, 800000, 100),
    'quality': np.random.uniform(1, 4, 100), 
    'price': np.random.uniform(300000, 900000, 100)
})


In [ ]:
X = df.drop(columns=['price']).values   # input
y = df['price'].values   # output

In [ ]:
# Scale of characteristics
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Price Model
price_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
price_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Model training
price_model.fit(X_scaled, y, epochs=10, batch_size=8)

In [ ]:
# Draw the Diagram
plot_model(price_model, to_file='price_model_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Save Model
price_model.save("price_model.h5")

In [ ]:
# Loading Quality Model
quality_model = load_model("quality_model.h5")

In [ ]:
# Loading a House Image
img = load_img("house1.jpg", target_size=(128, 128))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
# Prediction Of Quality
predicted_quality = quality_model.predict(img_array)[0][0]

In [ ]:
# Other characteristics of the house
features = {
    'bedrooms': 3,
    'bathrooms': 2,
    'solar': 1,
    'AC': 1,
    'pool': 0,
    'avg_nearby_price': 450000,
    'quality': predicted_quality
}


In [ ]:
# Data Preparation
input_data = np.array([list(features.values())])
scaler = StandardScaler()
input_scaled = scaler.fit_transform(input_data)

In [ ]:
# Download the price and prediction model
price_model = load_model("price_model.h5")
predicted_price = price_model.predict(input_scaled)[0][0]

In [ ]:
print(f"Estimated Property Value: ${predicted_price:,.2f}")